# OPEN-CV PROJECT: LIS

# Import all dependencies

In [1]:
import cv2
import time
import mediapipe as mp
#import tensorflow
import numpy as np
import os
from matplotlib import pyplot as plt

#  Holistic model

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [37]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    
def draw_landmarks_custom(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(0,0,255),thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(0,0,255),thickness=1, circle_radius=1),
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,110,10),thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121),thickness=1, circle_radius=1),
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255,0,0),thickness=3, circle_radius=5),
                             mp_drawing.DrawingSpec(color=(255,0,0),thickness=3, circle_radius=5),
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0,0,255),thickness=3, circle_radius=5),
                             mp_drawing.DrawingSpec(color=(0,0,255),thickness=3, circle_radius=5),
                             )
    cv2.putText(frame, 'Left Hand', (15,132), cv2.FONT_HERSHEY_SIMPLEX, 1 ,(255,0,0), 4, cv2.LINE_AA) 
    cv2.putText(frame, 'Right Hand', (15,232), cv2.FONT_HERSHEY_SIMPLEX, 1 ,(0,0,255), 4, cv2.LINE_AA) 



In [36]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, 
                          min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #black = np.zeros((1080, 1920, 3))
        ret, frame = cap.read()
        #frame = cv2.flip(frame,1)
        
        # make detection
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        draw_landmarks_custom(frame, results)
        
        cv2.imshow('LIS', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

# Extract point for training and prediction

In [38]:
def points_detection(results):
    rh = np.array([[points.x, points.y, points.z] for points in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh = np.array([[points.x, points.y, points.z] for points in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    po = np.array([[points.x, points.y, points.z] for points in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(99)
    return np.concatenate([lh, rh, po])

# Setup Folders for dataset

In [39]:
DATA_PATH = os.path.join('LIS_data')
labels = np.array(['a', 'b', 'c']) # put the entire alphabet in the future
no_sequences = 101 # the first one will be skipped
sequence_lenght = 30

In [40]:
#for label in labels:
#    for sequence in range(no_sequences):
#        try:
#            os.makedirs(os.path.join(DATA_PATH, label, str(sequence)))
#        except:
#            pass

# Collect data for training

In [43]:
data = []

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for label in labels:
        for id in range(no_sequences):
    
            ret, frame = cap.read()

            # make detection
            image, results = mediapipe_detection(frame, holistic)
            #print(results)

            draw_landmarks_custom(frame, results)

            if id == 0:
                cv2.putText(frame, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1 ,(0,255,0), 4, cv2.LINE_AA)
                #cv2.putText(frame, f'Collecting frame {id} for {label}', (15,32), cv2.FONT_HERSHEY_SIMPLEX, 1 ,(0,255,0), 4, cv2.LINE_AA)
                cv2.imshow('LIS', frame)
                cv2.waitKey(2000)
            else:
                data.append(points_detection(results))
                cv2.putText(frame, f'Collecting frame {id} for {label}', (15,32), cv2.FONT_HERSHEY_SIMPLEX, 1 ,(0,255,0), 4, cv2.LINE_AA) 
                cv2.waitKey(100)
                cv2.imshow('LIS', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [14]:
import pandas as pd
df =pd.read_csv('data.csv')
df = df.iloc[:,1:]

In [44]:
import pandas as pd
df = pd.DataFrame(np.array(data))
y=[]
for i in labels:
    y = np.concatenate([y, [i] * (no_sequences-1)])
df['y'] = y
pd.DataFrame(df).to_csv('data.csv')
df


,0,1,2,3,4,5,6,7,8,9,...,216,217,218,219,220,221,222,223,224,y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.494195,2.527829,0.647707,0.674836,2.610976,0.071677,0.530219,2.634062,0.225317,a
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.469672,2.570124,0.545969,0.661475,2.659776,0.073329,0.509595,2.673747,0.114094,a
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.461516,2.633976,0.574484,0.648224,2.722558,0.054093,0.503542,2.751169,0.152413,a
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.457963,2.633293,0.583292,0.644880,2.718938,-0.092633,0.498846,2.749284,0.161315,a
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.438489,2.644746,0.607440,0.634049,2.731106,-0.058439,0.477038,2.761000,0.168193,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.524335,3.041526,0.310595,0.723051,3.118100,-0.074054,0.569455,3.147402,-0.217166,c
296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.524155,3.037609,0.139592,0.724839,3.115733,-0.078892,0.568951,3.142724,-0.388451,c
297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.518385,3.037474,0.060310,0.719661,3.115462,-0.121105,0.563638,3.140680,-0.456102,c
298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.498612,3.037480,0.105792,0.695450,3.121368,-0.111285,0.545124,3.143309,-0.407078,c


# Model

In [45]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
import datetime
from datetime import datetime

In [46]:
model_name = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

tensorboard = TensorBoard(
    log_dir="logs/scalars/" + model_name,
    histogram_freq=0,
    write_graph=True,
    write_images=True,
    profile_batch=0)

In [47]:
label_map = {label:num for num, label in enumerate(labels)}
label_map

{'a': 0, 'b': 1, 'c': 2}

In [48]:
# split into input (X) and output (y) variables
X = np.array(df.iloc[:,0:-1])
y = np.array(df.iloc[:,-1])

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)

#print(integer_encoded)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

# invert first example
#inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
#print(inverted)

In [98]:
# define the keras model
model = Sequential()
model.add(Dense(32, input_dim=225, activation='relu'))
model.add(Dense(16, activation='relu'))
#model.add(Dense(16, activation='relu'))
#model.add(Dense(16, activation='relu'))
model.add(Dense(labels.shape[0], activation='softmax'))

In [99]:
# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [100]:
# fit the keras model on the dataset
model.fit(X, y, epochs=100, batch_size=20, callbacks=[tensorboard], validation_split=0.2)

Train on 240 samples, validate on 60 samples
Epoch 1/100
240/240 [==============================] - 0s 604us/step - loss: 1.0801 - categorical_accuracy: 0.4583 - val_loss: 1.6566 - val_categorical_accuracy: 0.0000e+00
Epoch 2/100
240/240 [==============================] - 0s 218us/step - loss: 0.9992 - categorical_accuracy: 0.4833 - val_loss: 1.6757 - val_categorical_accuracy: 0.0000e+00
Epoch 3/100
240/240 [==============================] - 0s 545us/step - loss: 0.9538 - categorical_accuracy: 0.5167 - val_loss: 1.6526 - val_categorical_accuracy: 0.0000e+00
Epoch 4/100
240/240 [==============================] - 0s 213us/step - loss: 0.9180 - categorical_accuracy: 0.6292 - val_loss: 1.7543 - val_categorical_accuracy: 0.0000e+00
Epoch 5/100
240/240 [==============================] - 0s 366us/step - loss: 0.8804 - categorical_accuracy: 0.7583 - val_loss: 1.5253 - val_categorical_accuracy: 0.0000e+00
Epoch 6/100
240/240 [==============================] - 0s 217us/step - loss: 0.8471 - cate

Epoch 49/100
240/240 [==============================] - 0s 134us/step - loss: 0.0865 - categorical_accuracy: 0.9917 - val_loss: 0.2159 - val_categorical_accuracy: 1.0000
Epoch 50/100
240/240 [==============================] - 0s 116us/step - loss: 0.0722 - categorical_accuracy: 0.9875 - val_loss: 0.1801 - val_categorical_accuracy: 1.0000
Epoch 51/100
240/240 [==============================] - 0s 109us/step - loss: 0.0685 - categorical_accuracy: 0.9958 - val_loss: 0.4231 - val_categorical_accuracy: 0.8667
Epoch 52/100
240/240 [==============================] - 0s 112us/step - loss: 0.0708 - categorical_accuracy: 0.9958 - val_loss: 0.3486 - val_categorical_accuracy: 0.9333
Epoch 53/100
240/240 [==============================] - 0s 182us/step - loss: 0.0702 - categorical_accuracy: 0.9917 - val_loss: 0.1257 - val_categorical_accuracy: 1.0000
Epoch 54/100
240/240 [==============================] - 0s 128us/step - loss: 0.0596 - categorical_accuracy: 0.9958 - val_loss: 0.2036 - val_categoric

240/240 [==============================] - 0s 110us/step - loss: 0.0108 - categorical_accuracy: 1.0000 - val_loss: 0.0556 - val_categorical_accuracy: 1.0000
Epoch 98/100
240/240 [==============================] - 0s 105us/step - loss: 0.0107 - categorical_accuracy: 1.0000 - val_loss: 0.0557 - val_categorical_accuracy: 1.0000
Epoch 99/100
240/240 [==============================] - 0s 123us/step - loss: 0.0102 - categorical_accuracy: 1.0000 - val_loss: 0.0419 - val_categorical_accuracy: 1.0000
Epoch 100/100
240/240 [==============================] - 0s 99us/step - loss: 0.0095 - categorical_accuracy: 1.0000 - val_loss: 0.0465 - val_categorical_accuracy: 1.0000


In [91]:
labels[np.argmax(model.predict(X)[233])]

'c'

# Save models

In [102]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


# Evaluate model

In [92]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [93]:
yhat = model.predict(X)
ytrue = np.argmax(y, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [94]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[200,   0],
        [  0, 100]],

       [[200,   0],
        [  0, 100]],

       [[200,   0],
        [  0, 100]]])

In [95]:
accuracy_score(ytrue, yhat)

1.0

# Testing

In [104]:
from keras.models import model_from_json

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [105]:
threshold = 0.1 # not used for the moment

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, 
                          min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        #frame = cv2.flip(frame,1)
        # make detection
        image, results = mediapipe_detection(frame, holistic)
        
        #print(results)
        
        prediction = labels[np.argmax(model.predict(np.array([points_detection(results)])))]
        pred_prob = int(np.max(model.predict(np.array([points_detection(results)])))*100)        
        
        if pred_prob > int(threshold*100):
            cv2.putText(frame, f'Lettera: {prediction} ({pred_prob}%)', 
                        (120,200), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        2 ,
                        (0,255,0), 
                        4, 
                        cv2.LINE_AA)
        else:
            cv2.putText(frame, f'Lettera: Uknown', 
                        (120,200), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        2 ,
                        (0,255,0), 
                        4, 
                        cv2.LINE_AA)

        #frame = cv2.flip(frame,1)

            

        #draw_landmarks_custom(frame, results)
  
        cv2.imshow('LIS', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         

...

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         